In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
llm = Ollama(model="llama2")
embeddings_provider = OllamaEmbeddings()

In [3]:
loader = WebBaseLoader(
    ["https://en.wikipedia.org/wiki/2024_Summer_Olympics"
     ]
)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

In [5]:
vector_store = DocArrayInMemorySearch.from_documents(document_chunks, embeddings_provider)

In [6]:
template = """Answer the question based only on the following context:
                {context}

                Question: {question}
                """
prompt = ChatPromptTemplate.from_template(template)

In [7]:
retriever = vector_store.as_retriever(
                search_type="similarity"
            )
output_parser = StrOutputParser()

In [8]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser


In [9]:
result = chain.invoke("Where are the 2024 summer olympics being held?")
print(result)

The 2024 Summer Olympics are being held in Paris, France. The opening ceremony took place on July 26, 2024, at the Stade de France, and the games are being hosted simultaneously in both Paris and Los Angeles.


In [10]:
result = chain.invoke("What are the new sports that are being added for the 2024 summer olympics?")
print(result)

The 2024 Summer Olympics will feature several new sports and disciplines, including:

1. Breakdancing: This urban dance style has been added to the Olympic program after a successful demonstration event at the 2020 Summer Olympics in Tokyo.
2. Skateboarding: Skateboarding will make its Olympic debut in Paris, with events for men and women in the street and park categories.
3. Surfing: Surfing will also make its Olympic debut in Paris, with events for men and women in the shortboard and longboard categories.
4. Sport Climbing: Sport climbing will be included in the 2024 Olympics, with events for men and women in the bouldering, lead, and speed disciplines.
5. Karate: Karate will make its Olympic debut in Paris, with events for both men and women in the kumite and kata categories.
6. Skateboarding and BMX Freestyle: These two sports will be included in the 2024 Olympics as demonstration events, with the possibility of becoming full medal events in the future.
7. Rugby: Rugby sevens will 